In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA,TruncatedSVD,NMF
from sklearn.preprocessing import Normalizer
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

In [2]:
heads=pd.read_csv("/data/dharp/compounding/datasets/heads_reduced.csv",sep="\t")
heads=heads.query('decade != 2000')
heads.columns=['joiner','decade','context','count']
heads['joiner']=heads['joiner'].str.replace(r'_n$', r'_h', regex=True)
heads=heads.groupby(['joiner','decade','context'])['count'].sum()
display(heads.shape)
heads.head()

(27727848,)

joiner  decade  context 
a_h     1800    a_a          4
                a_n         16
                aa_n         4
                act_v        2
                almost_r     3
Name: count, dtype: int64

In [3]:
modifiers=pd.read_csv("/data/dharp/compounding/datasets/modifiers_reduced.csv",sep="\t")
modifiers=modifiers.query('decade != 2000')
modifiers.columns=['joiner','decade','context','count']
modifiers['joiner']=modifiers['joiner'].str.replace(r'_n$', r'_m', regex=True)
modifiers=modifiers.groupby(['joiner','decade','context'])['count'].sum()
display(modifiers.shape)
modifiers.head()

(23495976,)

joiner  decade  context
a_m     1800    act_n       1
                b_n        24
                be_v       24
                c_n        16
                call_v      1
Name: count, dtype: int64

In [4]:
compounds=pd.read_csv("/data/dharp/compounding/datasets/compounds_reduced.csv",sep="\t",index_col=0,usecols=[0,3,4,5,6])
compounds=compounds.query('decade != 2000')
compounds.columns=['decade','context','count','joiner']
compounds=compounds.groupby(['joiner','decade','context'])['count'].sum()
display(compounds.shape)
compounds.head()

/data/dharp/packages/miniconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(35677592,)

joiner         decade  context     
a_n aaaaa_n    1960    a_n              9
               1970    a_n             10
               1980    a_n              9
               1990    a_n             14
a_n addison_n  1960    publishing_n    14
Name: count, dtype: int64

In [ ]:
df=pd.concat([heads,modifiers,compounds])

In [ ]:
df=df.to_sparse()

In [ ]:
df, rows, columns = df.to_coo(row_levels=['joiner','decade'],column_levels=['context'],sort_labels=False)

In [ ]:
len(columns)

In [ ]:
df_svd = TruncatedSVD(n_components=300, n_iter=10, random_state=1991)
display(df_svd.fit(df).explained_variance_ratio_.sum())
df_list=df_svd.fit(df).explained_variance_ratio_
df_reduced = df_svd.fit_transform(df)
df_reduced = Normalizer(copy=False).fit_transform(df_reduced)
df_reduced=pd.DataFrame(df_reduced,index=rows)
df_reduced.reset_index(inplace=True)
df_reduced

In [ ]:
df_reduced.index = pd.MultiIndex.from_tuples(df_reduced.index, names=['joiner', 'decade'])

In [ ]:
pd.Series(constituents_list).cumsum().plot(figsize=[15,15])

In [12]:
co_occ_reduced.reset_index(inplace=True)

In [15]:
heads=co_occ_reduced.loc[co_occ_reduced.constituent.str.endswith(r'_h')]
heads['constituent']=heads['constituent'].str.replace(r'_h$', r'_n', regex=True)
heads.to_csv('/data/dharp/compounding/datasets/heads_context_decadal_svd.csv',index=False,header=False,sep='\t')

In [16]:
modifiers=co_occ_reduced.loc[co_occ_reduced.constituent.str.endswith(r'_m')]
modifiers['constituent']=modifiers['constituent'].str.replace(r'_m$', r'_n', regex=True)
modifiers.to_csv('/data/dharp/compounding/datasets/modifiers_context_decadal_svd.csv',index=False,header=False,sep='\t')